In [2]:
import json
import time
import requests
import pandas as pd
import re
import string

In [3]:
# 과학적 표기법 사용안함
pd.options.display.float_format = '{:.2f}'.format

In [4]:
# API로 데이터 불러오기
start_time = time.time()
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
limit = 0
api_url = f'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit={limit}'
api = requests.get(api_url, headers=headers)
print(f"{time.time()-start_time} s")

4.325690984725952 s


In [5]:
start_time = time.time()
# deserialization
data = json.loads(api.text)
print(f"{time.time()-start_time} s")

0.04660820960998535 s


In [6]:
# 상장 기업 수 체크
start_time = time.time()
num_of_listed_comp = data['data']['totalrecords']
print(f"{time.time()-start_time} s")

0.0001270771026611328 s


In [7]:
start_time = time.time()
num_of_listed_comp = data.get('data').get('totalrecords') # which is faster?
print(f"{time.time()-start_time} s")

9.703636169433594e-05 s


In [8]:
print(num_of_listed_comp)

7798


In [9]:
# 상장 기업 리스트
d = data['data']['table']['rows']

In [212]:
start_time = time.time()
df = pd.DataFrame(d)
print(f"{time.time()-start_time} s")

0.014223814010620117 s


In [213]:
comp_list = df.iloc[:,:2]
comp_list.head()

,symbol,name
0,AAPL,Apple Inc. Common Stock
1,MSFT,Microsoft Corporation Common Stock
2,AMZN,"Amazon.com, Inc. Common Stock"
3,GOOG,Alphabet Inc. Class C Capital Stock
4,GOOGL,Alphabet Inc. Class A Common Stock


In [215]:
p="""
r'\,|\s\(([\w].*)\)*|\sClass\s[\w].*|(\bCommon\b).*|(\sNew)?(\s[\w]*)?(\sStock|\sShare|\sADR).*|\sAmerican\s(Depositary|Depository).*'
r'\,|\s\(([\w].*)\)*|(\bcommon\b|\bclass\b).*|(\sNew)?(\s[\w]*)?(\sstock|\sshare|(\sADR|\sADS)).*|([\w]*)(\sdepository|\sdepositary).*'
r'\,|\s\(([\w].*)\)*|\bcommon\b.*|(\sNew)?(\s[\w]*)?(\sStock|\sShare).*|(american.+[\w].share+s?).*'
\,|\s\(([\w].*)\)*|(\snew)?(\s[\w]*)?(\sstock|\sshare).*|((?<=inc\.)|(?<=inc))\s(\w|\W).*
"""

In [216]:
pattern = r'\,|\s\(([\w].*)\)*|(\snew)?(\s[\w]*)?(\sstock|\sshare).*|((?<=inc\.)|(?<=inc))\s(\w|\W).*'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [217]:
start_time = time.time()
for index, row in comp_list.iterrows():
    try:
        # 중복 단어 제거
        #words = row['name'].split()
        #row['name'] = ' '.join(sorted(set(words), key=words.index))
        
        # 필요 없는 문자 제거
        row['name'] = regex.sub('', row['name'])
    except Exception as e:
        print(e)
        break;
print(f"{time.time()-start_time} s")

1.553912878036499 s


In [224]:
len(comp_list)

In [218]:
# 없어도 됨
for i in comp_list.values:
    print(i)

['AAPL' 'Apple Inc.']
['MSFT' 'Microsoft Corporation']
['AMZN' 'Amazon.com Inc.']
['GOOG' 'Alphabet Inc.']
['GOOGL' 'Alphabet Inc.']
['FB' 'Facebook Inc.']
['TSM' 'Taiwan Semiconductor Manufacturing Company Ltd.']
['BABA' 'Alibaba Group Holding Limited American']
['TSLA' 'Tesla Inc.']
['JPM' 'JP Morgan Chase & Co.']
['V' 'Visa Inc.']
['JNJ' 'Johnson & Johnson']
['NVDA' 'NVIDIA Corporation']
['WMT' 'Walmart Inc.']
['UNH' 'UnitedHealth Group Incorporated']
['BAC' 'Bank of America Corporation']
['MA' 'Mastercard Incorporated']
['HD' 'Home Depot Inc.']
['PG' 'Procter & Gamble Company']
['DIS' 'Walt Disney Company']
['PYPL' 'PayPal Holdings Inc.']
['ASML' 'ASML Holding N.V. New York']
['CMCSA' 'Comcast Corporation Class A']
['XOM' 'Exxon Mobil Corporation']
['TM' 'Toyota Motor Corporation']
['KO' 'Coca-Cola Company']
['ADBE' 'Adobe Inc.']
['VZ' 'Verizon Communications Inc.']
['INTC' 'Intel Corporation']
['ORCL' 'Oracle Corporation']
['CSCO' 'Cisco Systems Inc.']
['NFLX' 'Netflix Inc.']
['PF

['HGV' 'Hilton Grand Vacations Inc.']
['MTSI' 'MACOM Technology Solutions Holdings Inc.']
['BOX' 'Box Inc.']
['OGS' 'ONE Gas Inc.']
['SHAK' 'Shake Shack Inc.']
['ALGT' 'Allegiant Travel Company']
['SAGE' 'Sage Therapeutics Inc.']
['MGNI' 'Magnite Inc.']
['AEIS' 'Advanced Energy Industries Inc.']
['MTH' 'Meritage Homes Corporation']
['VIAV' 'Viavi Solutions Inc.']
['SIX' 'Six Flags Entertainment Corporation']
['MDC' 'M.D.C. Holdings Inc.']
['ALRM' 'Alarm.com Holdings Inc.']
['BECN' 'Beacon Roofing Supply Inc.']
['DOC' 'Physicians Realty Trust']
['SAVE' 'Spirit Airlines Inc.']
['ACVA' 'ACV Auctions Inc.']
['PCH' 'PotlatchDeltic Corporation']
['SANA' 'Sana Biotechnology Inc.']
['PAY' 'Paymentus Holdings Inc.']
['ENV' 'Envestnet Inc']
['COKE' 'Coca-Cola Consolidated Inc.']
['DAN' 'Dana Incorporated']
['SBRA' 'Sabra Health Care REIT Inc.']
['FELE' 'Franklin Electric Co. Inc.']
['ABCB' 'Ameris Bancorp']
['AJRD' 'Aerojet Rocketdyne Holdings Inc.']
['VSAT' 'ViaSat Inc.']
['TMHC' 'Taylor Morris

['WRLD' 'World Acceptance Corporation']
['PETQ' 'PetIQ Inc.']
['CET' 'Central Securities Corporation']
['CCO' 'Clear Channel Outdoor Holdings Inc.']
['NEX' 'NexTier Oilfield Solutions Inc.']
['PCVX' 'Vaxcyte Inc.']
['OCUL' 'Ocular Therapeutix Inc.']
['BHE' 'Benchmark Electronics Inc.']
['MX' 'Magnachip Semiconductor Corporation']
['ALTM' 'Altus Midstream Company Class A']
['SRG' 'Seritage Growth Properties Class A']
['GABC' 'German American Bancorp Inc.']
['AGM' 'Federal Agricultural Mortgage Corporation']
['QUOT' 'Quotient Technology Inc.']
['DENN' "Denny's Corporation"]
['HQH' 'Tekla Healthcare Investors']
['APR' 'Apria Inc.']
['DHT' 'DHT Holdings Inc.']
['RMAX' 'RE/MAX Holdings Inc.']
['PLL' 'Piedmont Lithium Inc.']
['BOOM' 'DMC Global Inc.']
['CLS' 'Celestica Inc.']
['RC' 'Ready Capital Corproation']
['AIV' 'Apartment Investment and Management Company']
['ICLK' 'iClick Interactive Asia Group Limited American']
['RPT' 'RPT Realty']
['MNRL' 'Brigham Minerals Inc.']
['HZO' 'MarineMax 

['ALUS' 'Alussa Energy Acquisition Corp. Class A']
['DHBCU' 'DHB Capital Corp. Unit']
['JFU' '9F Inc.']
['BLUA' 'BlueRiver Acquisition Corp. Class A']
['BYRN' 'Byrna Technologies Inc.']
['EMKR' 'EMCORE Corporation']
['WSR' 'Whitestone REIT']
['KIII' 'Kismet Acquisition Three Corp. Class A']
['ENFA' '890 5th Avenue Partners Inc.']
['PVAC' 'Penn Virginia Corporation']
['DGNS' 'Dragoneer Growth Opportunities Corp. II Class A']
['MESA' 'Mesa Air Group Inc.']
['TZPS' 'TZP Strategies Acquisition Corp. Class A']
['VLGEA' 'Village Super Market Inc.']
['MSON' 'MISONIX Inc.']
['SGTX' 'Sigilon Therapeutics Inc.']
['SURF' 'Surface Oncology Inc.']
['BDSI' 'BioDelivery Sciences International Inc.']
['PCQ' 'PIMCO California Municipal Income Fund']
['PSF' 'Cohen & Steers Select Preferred and Income Fund Inc.']
['AMPI' 'Advanced Merger Partners Inc.']
['HYI' 'Western Asset High Yield Defined Opportunity Fund Inc.']
['FORA' 'Forian Inc.']
['DNZ' 'D and Z Media Acquisition Corp. Class A']
['TUEM' 'Tuesda

['MCAD' 'Mountain Crest Acquisition Corp. II']
['LIFE' 'aTyr Pharma Inc.']
['AAU' 'Almaden Minerals Ltd.']
['AUVI' 'Applied UV Inc.']
['KBNT' 'Kubient Inc.']
['GBRG' 'Goldenbridge Acquisition Limited']
['DIT' 'AMCON Distributing Company']
['KTCC' 'Key Tronic Corporation']
['VERB' 'Verb Technology Company Inc.']
['CHEK' 'Check-Cap Ltd.']
['FCO' 'Aberdeen Global Income Fund Inc.']
['DFFN' 'Diffusion Pharmaceuticals Inc.']
['UG' 'United-Guardian Inc.']
['BCAC' 'Brookline Capital Acquisition Corp.']
['LINK' 'Interlink Electronics Inc.']
['HTY' 'John Hancock Tax-Advantaged']
['SDH' 'Global Internet of People Inc.']
['IROQ' 'IF Bancorp Inc.']
['EVFM' 'Evofem Biosciences Inc.']
['BLUW' 'Blue Water Acquisition Corp. Class A']
['ADXN' 'Addex Therapeutics Ltd American']
['MVO' 'MV Oil Trust Units of Beneficial Interests']
['ICCC' 'ImmuCell Corporation']
['BCDA' 'BioCardia Inc.']
['DDMX' 'DD3 Acquisition Corp. II Class A']
['DCTH' 'Delcath Systems Inc.']
['NEON' 'Neonode Inc.']
['VBFC' 'Village B

['LEXXW' 'Lexaria Bioscience Corp. Warrant']
['LFT^A' 'Lument Finance Trust Inc.']
['LFTRU' 'Lefteris Acquisition Corp. Unit']
['LFTRW' 'Lefteris Acquisition Corp. Warrant']
['LGACU' 'Lazard Growth Acquisition Corp. I Units']
['LGACW' 'Lazard Growth Acquisition Corp. I Warrants']
['LGHLW' 'Lion Group Holding Ltd. Warrant']
['LIV' 'Emles @Home ETF']
['LIVKU' 'LIV Capital Acquisition Corp. Unit']
['LIVKW' 'LIV Capital Acquisition Corp. Warrant']
['LIXTW' 'Lixte Biotechnology Holdings Inc.']
['LJAQU' 'LightJump Acquisition Corporation Unit']
['LJAQW' 'LightJump Acquisition Corporation Warrant']
['LMACU' 'Liberty Media Acquisition Corporation Unit']
['LMACW' 'Liberty Media Acquisition Corporation Warrants']
['LMAOU' 'LMF Acquisition Opportunities Inc.']
['LMAOW' 'LMF Acquisition Opportunities Inc.']
['LMRKN'
 'Landmark Infrastructure Partners LP 7% Series C Fltg/Fxd Perpetual Conv']
['LMRKO'
 'Landmark Infrastructure Partners LP Perpetual Preferred Units Series B 7.90%']
['LMRKP'
 'Landmar

In [202]:
for index, row in df.iterrows():
    # %가 포함 종목 제거
    if re.search(r'\%',row['name']):
        df = df.drop(index=index)

In [220]:
# 없어도 됨
for i in comp_list.values:
    if(i[0] == 'PAC'):
        print(f"{i[0]} | {i[1]}")

PAC | Grupo Aeroportuario Del Pacifico S.A. B. de C.V. Grupo Aeroportuario Del Pacifico S.A. de C.V.


In [211]:
s = "Johnson & Johnson Common Stock"
re.sub(r'(\snew)?(\s[\w]*)?(\sstock|\sshare).*|((?<=inc\.)|(?<=inc))\s(\w|\W).*', '', s, flags=re.IGNORECASE)

'Johnson & Johnson'